# Read data from DynamoDB table

## Import modules

In [1]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import yaml
import json
import pandas as pd

## Variables

In [2]:
TABLENAME='simpsonsEpisodes'

## Get credentials

In [3]:
# Load credentials from YAML file
with open("credentials.yml", "r") as keyholder:
    try:
        credentials=yaml.safe_load(keyholder)
    except yaml.YAMLError as exc:
        print(exc)

In [4]:
AWS_SERVER_PUBLIC_KEY=credentials['dynamodb']['AWS_SERVER_PUBLIC_KEY']
AWS_SERVER_SECRET_KEY=credentials['dynamodb']['AWS_SERVER_SECRET_KEY']
REGION_NAME=credentials['dynamodb']['REGION_NAME']

## DynamoDB client and resource

### Create boto3 client for DynamoDb

In [5]:
dynamodbClient = boto3.client(
    "dynamodb",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_SERVER_PUBLIC_KEY,
    aws_secret_access_key=AWS_SERVER_SECRET_KEY,
)

In [6]:
dynamodbClient

### Create boto3 resource for DynamoDb

In [7]:
dynamodbRes = boto3.resource(
    "dynamodb",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_SERVER_PUBLIC_KEY,
    aws_secret_access_key=AWS_SERVER_SECRET_KEY,
)

In [8]:
dynamodbRes

dynamodb.ServiceResource()

## Get item

### Get item with client

In [9]:
response = dynamodbClient.get_item(
    TableName=TABLENAME,
    Key={
        'No_Season': {'N': '1'},
        'No_Inseason': {'N': '8'}
    }
)

In [10]:
print(response['Item'])

{'Title': {'S': 'The Telltale Head'}, 'ProdCode': {'S': '7G07'}, 'OriginalAirDate': {'S': '1990-02-25'}, 'USViewers(millions)': {'N': '28'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['Rich Moore']}, 'No_Overall': {'N': '8'}, 'WrittenBy': {'SS': ['Al Jean', 'Matt Groening', 'Mike Reiss', 'Sam Simon']}, 'No_Inseason': {'N': '8'}}


### Get item with resource

In [11]:
table = dynamodbRes.Table(TABLENAME)

response = table.get_item(
    Key={
        'No_Season': 1,
        'No_Inseason': 8
    }
)


In [12]:
print(response['Item'])

{'Title': 'The Telltale Head', 'ProdCode': '7G07', 'OriginalAirDate': '1990-02-25', 'USViewers(millions)': Decimal('28'), 'No_Season': Decimal('1'), 'DirectedBy': {'Rich Moore'}, 'No_Overall': Decimal('8'), 'WrittenBy': {'Mike Reiss', 'Matt Groening', 'Al Jean', 'Sam Simon'}, 'No_Inseason': Decimal('8')}


### Get item with resource and transform it it into a Pandas Data Frame

In [13]:
df = pd.DataFrame.from_dict(response['Item'], orient='index')
df=df.T
df.head()

,Title,ProdCode,OriginalAirDate,USViewers(millions),No_Season,DirectedBy,No_Overall,WrittenBy,No_Inseason
0,The Telltale Head,7G07,1990-02-25,28,1,{Rich Moore},8,"{Mike Reiss, Matt Groening, Al Jean, Sam Simon}",8


## Query data from index

In [14]:
table = dynamodbRes.Table(TABLENAME)

response = table.query(
    IndexName="AirDateIndex",
    KeyConditionExpression=Key('OriginalAirDate').eq('1990-02-25'),
)

In [15]:
print(response['Items'])

[{'Title': 'The Telltale Head', 'ProdCode': '7G07', 'OriginalAirDate': '1990-02-25', 'USViewers(millions)': Decimal('28'), 'DirectedBy': {'Rich Moore'}, 'No_Season': Decimal('1'), 'No_Overall': Decimal('8'), 'WrittenBy': {'Mike Reiss', 'Matt Groening', 'Al Jean', 'Sam Simon'}, 'No_Inseason': Decimal('8')}]


## Query data

### Query HASH and SORT keys: equals

In [16]:
response = dynamodbClient.query(
    TableName=TABLENAME,
    KeyConditionExpression='No_Season = :Season AND No_Inseason = :Inseason',
    ExpressionAttributeValues={
        ':Season': {'N': '1'},
        ':Inseason': {'N': '8'}
    }
)

In [17]:
print(response['Items'])

[{'Title': {'S': 'The Telltale Head'}, 'ProdCode': {'S': '7G07'}, 'OriginalAirDate': {'S': '1990-02-25'}, 'USViewers(millions)': {'N': '28'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['Rich Moore']}, 'No_Overall': {'N': '8'}, 'WrittenBy': {'SS': ['Al Jean', 'Matt Groening', 'Mike Reiss', 'Sam Simon']}, 'No_Inseason': {'N': '8'}}]


### Query HASH key: equals

In [18]:
response = dynamodbClient.query(
    TableName=TABLENAME,
    KeyConditionExpression='No_Season = :Season',
    ExpressionAttributeValues={
        ':Season': {'N': '1'}
    }
)

In [19]:
print(response['Items'])

[{'Title': {'S': 'Simpsons Roasting on an Open Fire'}, 'ProdCode': {'S': '7G08'}, 'OriginalAirDate': {'S': '1989-12-17'}, 'USViewers(millions)': {'N': '26.7'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['David Silverman']}, 'No_Overall': {'N': '1'}, 'WrittenBy': {'SS': ['Mimi Pond']}, 'No_Inseason': {'N': '1'}}, {'Title': {'S': 'Bart the Genius'}, 'ProdCode': {'S': '7G02'}, 'OriginalAirDate': {'S': '1990-01-14'}, 'USViewers(millions)': {'N': '24.5'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['David Silverman']}, 'No_Overall': {'N': '2'}, 'WrittenBy': {'SS': ['Jon Vitti']}, 'No_Inseason': {'N': '2'}}, {'Title': {'S': "Homer's Odyssey"}, 'ProdCode': {'S': '7G03'}, 'OriginalAirDate': {'S': '1990-01-21'}, 'USViewers(millions)': {'N': '27.5'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['Wes Archer']}, 'No_Overall': {'N': '3'}, 'WrittenBy': {'SS': ['Jay Kogen', 'Wallace Wolodarsky']}, 'No_Inseason': {'N': '3'}}, {'Title': {'S': "There's No Disgrace Like Home"}, 'ProdCode': {'S':

### Query HASH and SORT keys: between

In [20]:
response = dynamodbClient.query(
    TableName=TABLENAME,
    KeyConditionExpression='No_Season = :Season AND No_Inseason BETWEEN :InseasonStart AND :InseasonEnd',
    ExpressionAttributeValues={
        ':Season': {'N': '1'},
        ':InseasonStart': {'N': '1'},
        ':InseasonEnd': {'N': '8'}
    }
)

In [21]:
print(response['Items'])

[{'Title': {'S': 'Simpsons Roasting on an Open Fire'}, 'ProdCode': {'S': '7G08'}, 'OriginalAirDate': {'S': '1989-12-17'}, 'USViewers(millions)': {'N': '26.7'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['David Silverman']}, 'No_Overall': {'N': '1'}, 'WrittenBy': {'SS': ['Mimi Pond']}, 'No_Inseason': {'N': '1'}}, {'Title': {'S': 'Bart the Genius'}, 'ProdCode': {'S': '7G02'}, 'OriginalAirDate': {'S': '1990-01-14'}, 'USViewers(millions)': {'N': '24.5'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['David Silverman']}, 'No_Overall': {'N': '2'}, 'WrittenBy': {'SS': ['Jon Vitti']}, 'No_Inseason': {'N': '2'}}, {'Title': {'S': "Homer's Odyssey"}, 'ProdCode': {'S': '7G03'}, 'OriginalAirDate': {'S': '1990-01-21'}, 'USViewers(millions)': {'N': '27.5'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['Wes Archer']}, 'No_Overall': {'N': '3'}, 'WrittenBy': {'SS': ['Jay Kogen', 'Wallace Wolodarsky']}, 'No_Inseason': {'N': '3'}}, {'Title': {'S': "There's No Disgrace Like Home"}, 'ProdCode': {'S':

### Query HASH and SORT keys: less than

In [22]:
response = dynamodbClient.query(
    TableName=TABLENAME,
    KeyConditionExpression='No_Season = :Season AND No_Inseason < :Inseason',
    ExpressionAttributeValues={
        ':Season': {'N': '1'},
        ':Inseason': {'N': '10'}
    }
)

In [23]:
print(response['Items'])

[{'Title': {'S': 'Simpsons Roasting on an Open Fire'}, 'ProdCode': {'S': '7G08'}, 'OriginalAirDate': {'S': '1989-12-17'}, 'USViewers(millions)': {'N': '26.7'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['David Silverman']}, 'No_Overall': {'N': '1'}, 'WrittenBy': {'SS': ['Mimi Pond']}, 'No_Inseason': {'N': '1'}}, {'Title': {'S': 'Bart the Genius'}, 'ProdCode': {'S': '7G02'}, 'OriginalAirDate': {'S': '1990-01-14'}, 'USViewers(millions)': {'N': '24.5'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['David Silverman']}, 'No_Overall': {'N': '2'}, 'WrittenBy': {'SS': ['Jon Vitti']}, 'No_Inseason': {'N': '2'}}, {'Title': {'S': "Homer's Odyssey"}, 'ProdCode': {'S': '7G03'}, 'OriginalAirDate': {'S': '1990-01-21'}, 'USViewers(millions)': {'N': '27.5'}, 'No_Season': {'N': '1'}, 'DirectedBy': {'SS': ['Wes Archer']}, 'No_Overall': {'N': '3'}, 'WrittenBy': {'SS': ['Jay Kogen', 'Wallace Wolodarsky']}, 'No_Inseason': {'N': '3'}}, {'Title': {'S': "There's No Disgrace Like Home"}, 'ProdCode': {'S':

### Table scan

In [24]:
table = dynamodbRes.Table(TABLENAME)

response = table.scan(FilterExpression=Attr('Title').eq('The Princess Guide'))

In [25]:
print(response['Items'])

[{'Title': 'The Princess Guide', 'ProdCode': 'TABF08', 'OriginalAirDate': '2015-03-01', 'USViewers(millions)': Decimal('3.93'), 'No_Season': Decimal('26'), 'DirectedBy': {'Timothy Bailey'}, 'No_Overall': Decimal('567'), 'WrittenBy': {'Brian Kelley'}, 'No_Inseason': Decimal('15')}]
